In [1]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
# 定义数据库路径
db_path = r"Y:\YZ\plan.db"
db_path = "plan.db"
# 定义日期比较基准,上半年的就要看下一年的eps了，下半年的还看当年的。
cutoff_date = datetime.strptime("2025-06-30", "%Y-%m-%d")
# 查询语句
query = """
SELECT Symbol, Low_PE, High_PE, FEPS,FFEPS,YOYGrowth,EPSY,PEG,PEG5
FROM CValue
WHERE "FEPS" IS NOT NULL AND Low_PE IS NOT NULL AND High_PE IS NOT NULL;
"""

try:
    # 连接到数据库
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()

        # 检查数据库中是否存在表 CValue
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        if ('CValue',) not in tables:
            raise Exception("Table 'CValue' does not exist in the database.")

        # 执行查询
        cursor.execute(query)
        rows = cursor.fetchall()

        # 初始化结果和 symbol_list
        results = []
        symbol_list = []
        for row in rows:
            print (row)
            symbol, low_pe, high_pe, feps,ffeps,growth,epsy,PEG,PEG5 = row
            # 检查 EPSY 日期是否早于 cutoff_date
            use_ffeps = False
            if epsy:
                try:
                    epsy_date = datetime.strptime(epsy, "%Y-%m-%d")
                    if epsy_date <= cutoff_date:
                        use_ffeps = True
                        effective_epsy = epsy_date.replace(year=epsy_date.year + 1)
                    else:
                        effective_epsy= epsy_date
                except ValueError:
                    print(f"Invalid EPSY format for {symbol}: {epsy}. Skipping...")
                    continue

            # 根据条件选择 FEPS 或 FFEPS
            effective_eps = ffeps if use_ffeps else feps
            
            low_price = low_pe * effective_eps
            high_price = high_pe * effective_eps
            results.append({
                "Symbol": symbol,
                "LowPrice": low_price,
                "HighPrice": high_price,
                "Growth": growth,
                "effective_eps":effective_eps,
                "effective_epsy":effective_epsy,
                "PEG":PEG,
                "PEG5":PEG5,
            })
            symbol_list.append(symbol)  # 将 Symbol 添加到列表

        # 将结果转换为 DataFrame 并显示
        df_results = pd.DataFrame(results)
        print(df_results)  # 打印结果
        print("Symbol List:", symbol_list)  # 打印 symbol_list
except Exception as e:
    print("Error:", str(e))


('MSFT', 23.5, 31.5, 13.04785, 15.02583, 15.159432396908294, '2025-06-30', 2.1464322187281986, 2.3890377470158595)
('AAPL', 20.0, 30, 7.39374, 8.30857, 12.37303448592998, '2025-09-30', 2.6529519145633635, 3.4480110849166454)
('PYPL', 15.0, 25, 4.88661, 5.44368, 11.39992755714083, '2025-12-31', 1.5786166909006063, 1.5540687320172961)
('CAT', 11.2, 24.7, 22.07263, 24.25835, 9.902399487510097, '2025-12-31', 1.651949929542668, 1.7201123171078503)
('CVX', 10.0, 12, 11.13528, 12.94924, 16.290205544898736, '2025-12-31', 0.8285739487089022, 2.1983126927714007)
('V', 20.0, 30, 11.21488, 12.6671, 12.949046267102265, '2025-09-30', 2.152566415556874, 2.27354666460493)
('META', 16.9, 34.4, 25.39736, 28.86068, 13.636535450928756, '2025-12-31', 1.7633946617696319, 1.287980386673814)
('HON', 16.2, 22.4, 11.0249, 12.12845, 10.00961459967891, '2025-12-31', 1.995106701100992, 2.6139069585415857)
('UNH', 15.3, 24.6, 29.89301, 33.72617, 12.82293084570608, '2025-12-31', 1.3683749178223699, 1.171333574247996

In [3]:
import requests
from datetime import datetime
import pytz  # For timezone conversion
import time
%run constant.ipynb
import sqlite3
from datetime import datetime

# Function to insert lastPrice into the database
def update_last_price(data, db_path='plan.db'):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    try:
        for item in data:
            symbol = item.get('symbol')
            last_price = item.get('lastPrice')
            print (symbol,last_price)
            # 检查 symbol 是否存在于数据库中
            cursor.execute("""
                SELECT COUNT(*) FROM EMA169 WHERE symbol = ?
            """, (symbol,))
            exists = cursor.fetchone()[0]

            if exists:
                # 如果 symbol 存在，则更新 last_price
                cursor.execute("""
                    UPDATE EMA169
                    SET last_price = ?
                    WHERE symbol = ?
                """, (last_price, symbol))
            else:
                print(f"Symbol '{symbol}' does not exist in the EMA169 table and was not updated.")

        conn.commit()
        print("Last prices updated successfully.")
    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        conn.close()


# Set up Houston timezone
houston_tz = pytz.timezone('America/Chicago')

def convert_to_houston_time(timestamp):
    """Converts UNIX timestamp (milliseconds) to Houston time."""
    # Convert from milliseconds to seconds
    utc_time = datetime.utcfromtimestamp(timestamp / 1000)
    # Convert to Houston time
    houston_time = utc_time.replace(tzinfo=pytz.utc).astimezone(houston_tz)
    return houston_time.strftime('%Y-%m-%d %H:%M:%S')  # Format the time as a string

# Fetch current data for a single symbol
def get_current_data(symbol, access_token):
    base_url = f'https://api.schwabapi.com/marketdata/v1/{symbol}/quotes'
    headers = {
        'accept': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }
    
    response = requests.get(base_url, headers=headers)
    response.raise_for_status()  # Raise an error for bad responses
    data = response.json()
    
    #print (data)
    # Extract the needed data from the response
    symbol_data = data.get(symbol, {})
    last_price = symbol_data.get('regular', {}).get('regularMarketLastPrice', None)
    trade_time = symbol_data.get('regular', {}).get('regularMarketTradeTime', None)
    high_price = symbol_data.get('quote', {}).get('highPrice', None)
    low_price = symbol_data.get('quote', {}).get('lowPrice', None)
    open_price = symbol_data.get('quote', {}).get('openPrice', None)
    close_price = symbol_data.get('quote', {}).get('closePrice', None)
   
    # Convert trade_time to Houston time if available
    if trade_time:
        trade_time = convert_to_houston_time(trade_time)

    # Return a dictionary with the symbol, last price, and trade time
    return {'symbol': symbol, 'lastPrice': last_price,'high_price':high_price,'open_price':open_price,'close_price':close_price,'low_price':low_price,'datetime': trade_time}

# Fetch current data for a list of symbols
def get_current_data_for_symbols(symbols, access_token):
    results = []
    for symbol in symbols:
        # Fetch data for each symbol
        symbol_data = get_current_data(symbol, access_token)
        # Append the result to the list
        results.append(symbol_data)
        time.sleep(0.1)
    return results

# Example usage
access_token = get_new_tokens()  # Assuming you already have a function to get the access token
symbols = symbol_list  # List of symbols
current_price = get_current_data_for_symbols(symbols, access_token)

# Print the result
print(current_price)

if __name__ == "__main__":
    update_last_price(current_price, db_path='plan.db')


C:\Users\Rhino\AppData\Local\Temp\ipykernel_17160\3283846029.py:49: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(timestamp / 1000)


[{'symbol': 'MSFT', 'lastPrice': 418.95, 'high_price': 424.71, 'open_price': 424.63, 'close_price': 424.56, 'low_price': 415.02, 'datetime': '2025-01-10 15:00:00'}, {'symbol': 'AAPL', 'lastPrice': 236.85, 'high_price': 240.16, 'open_price': 240.01, 'close_price': 242.7, 'low_price': 233.0, 'datetime': '2025-01-10 15:00:00'}, {'symbol': 'PYPL', 'lastPrice': 83.36, 'high_price': 87.09, 'open_price': 86.79, 'close_price': 87.94, 'low_price': 82.24, 'datetime': '2025-01-10 15:00:00'}, {'symbol': 'CAT', 'lastPrice': 351.0, 'high_price': 359.85, 'open_price': 357.83, 'close_price': 361.07, 'low_price': 349.8, 'datetime': '2025-01-10 18:00:00'}, {'symbol': 'CVX', 'lastPrice': 153.14, 'high_price': 154.34, 'open_price': 153.41, 'close_price': 150.3, 'low_price': 150.87, 'datetime': '2025-01-10 18:00:00'}, {'symbol': 'V', 'lastPrice': 307.71, 'high_price': 311.85, 'open_price': 311.2, 'close_price': 312.6, 'low_price': 305.99, 'datetime': '2025-01-10 18:00:00'}, {'symbol': 'META', 'lastPrice': 

In [4]:
import sqlite3

def calculate_and_update_peg_from_cvalue():
    # Connect to the SQLite database
    db_path = "plan.db"
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    try:
        # Fetch all symbols from CValue
        cursor.execute("SELECT Symbol FROM CValue")
        symbols = [row[0] for row in cursor.fetchall()]

        for symbol in symbols:
            # Fetch last_price from EMA169 for the current symbol
            cursor.execute("SELECT last_price FROM EMA169 WHERE symbol = ?", (symbol,))
            row_ema169 = cursor.fetchone()

            if row_ema169 is None:
                print(f"Symbol {symbol} not found in EMA169.")
                continue

            last_price = row_ema169[0]

            if last_price is None or last_price <= 0:
                print(f"Invalid last_price value for symbol {symbol}.")
                continue

            # Fetch relevant data from CValue for the symbol
            cursor.execute("SELECT FEPS, YOYGrowth, Growth5Y FROM CValue WHERE Symbol = ?", (symbol,))
            row_cvalue = cursor.fetchone()

            if row_cvalue is None:
                print(f"Symbol {symbol} not found in CValue.")
                continue

            feps, yoy_growth, growth_5y = row_cvalue

            # Ensure data validity
            if feps is None or feps <= 0:
                print(f"Invalid FEPS value for symbol {symbol}.")
                continue

            if yoy_growth is None or yoy_growth <= 0:
                print(f"Invalid YOYGrowth value for symbol {symbol}.")
                continue

            if growth_5y is None or growth_5y <= 0:
                print(f"Invalid Growth5Y value for symbol {symbol}.")
                continue

            # Calculate PEG and PEG5
            peg = last_price / feps / yoy_growth
            peg5 = last_price / feps / growth_5y

            # Update the database with the calculated values
            cursor.execute(
                "UPDATE CValue SET PEG = ?, PEG5 = ? WHERE Symbol = ?",
                (peg, peg5, symbol)
            )
            print(f"PEG and PEG5 for symbol {symbol} updated successfully.")

        # Commit all updates to the database
        connection.commit()

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the database connection
        connection.close()

# Example usage
if __name__ == "__main__":
    calculate_and_update_peg_from_cvalue()


PEG and PEG5 for symbol MSFT updated successfully.
PEG and PEG5 for symbol AAPL updated successfully.
PEG and PEG5 for symbol PYPL updated successfully.
PEG and PEG5 for symbol CAT updated successfully.
PEG and PEG5 for symbol CVX updated successfully.
PEG and PEG5 for symbol V updated successfully.
PEG and PEG5 for symbol META updated successfully.
PEG and PEG5 for symbol HON updated successfully.
PEG and PEG5 for symbol UNH updated successfully.
PEG and PEG5 for symbol GS updated successfully.
PEG and PEG5 for symbol HD updated successfully.
PEG and PEG5 for symbol WMT updated successfully.
PEG and PEG5 for symbol GOOGL updated successfully.
PEG and PEG5 for symbol NFLX updated successfully.
PEG and PEG5 for symbol TSLA updated successfully.
PEG and PEG5 for symbol FDX updated successfully.
PEG and PEG5 for symbol MA updated successfully.
PEG and PEG5 for symbol ILMN updated successfully.
PEG and PEG5 for symbol TSM updated successfully.
PEG and PEG5 for symbol CF updated successfull

In [5]:
import sqlite3

def update_ema_notes(db_path):
    # 连接 SQLite 数据库
    connection = sqlite3.connect(db_path)
    cursor = connection.cursor()

    try:
        # 查询所有行
        cursor.execute("SELECT symbol, daily, weekly, monthly, last_price FROM EMA169")
        rows = cursor.fetchall()

        for row in rows:
            symbol, daily, weekly, monthly, last_price = row
            note = None

            # 条件 1: 完全空头排列的 EMA
            if daily < weekly < monthly:
                note = "short trend seq"

            # 条件 2: 下跌趋势
            if last_price < daily and last_price < weekly and last_price < monthly:
                if note:
                    note += "; down trend"
                else:
                    note = "down trend"
            if last_price > weekly:
                if note:
                    note += "; weekly up"
                else:
                    note = "weekly up"
            # 更新 note 列
            if note:
                cursor.execute("""
                    UPDATE EMA169
                    SET note = ?
                    WHERE symbol = ?
                """, (note, symbol))

        # 提交更改
        connection.commit()
        print("EMA notes updated successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # 关闭数据库连接
        connection.close()

# 示例用法
if __name__ == "__main__":
    update_ema_notes(db_path="plan.db")


EMA notes updated successfully.


In [6]:
import pandas as pd
import sqlite3

# CSV 文件路径
csv_file_path = "watch 2025-01-06.csv"

# SQLite 数据库路径
db_path = "plan.db" # 修改为实际数据库路径

# 表名
table_name = "CValue"

try:
    # 读取 CSV 文件
    data = pd.read_csv(csv_file_path)
    data_copy=data
    # 检查是否有 'Last Quarter Announce Date' 列
    if 'Last Quarter Announce Date' not in data.columns:
        raise KeyError("'Last Quarter Announce Date' column not found in the CSV file.")
    
    # 重命名列以匹配数据库字段
    data = data.rename(columns={"Last Quarter Announce Date": "ER_Date"})
    
    # 连接到 SQLite 数据库
    conn = sqlite3.connect(db_path)
    
    # 将数据更新到数据库
    for _, row in data.iterrows():
        symbol = row.get("Symbol", None)
        er_date = row.get("ER_Date", None)
        
        if symbol and er_date:
            # 使用 SQL 更新表
            query = f"""
            UPDATE {table_name}
            SET ER_Date = ?
            WHERE Symbol = ?;
            """
            conn.execute(query, (er_date, symbol))
    
    # 提交更改并关闭连接
    data = data_copy
    conn.commit()
    conn.close()
    print("Database updated successfully!")
    
except Exception as e:
    print(f"An error occurred: {e}")


Database updated successfully!


In [11]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

# 保持原有代码：获取和合并数据
today = datetime.today()
df_current = pd.DataFrame(current_price).rename(columns={'symbol': 'Symbol'})
df_merged = pd.merge(df_results, df_current[['Symbol', 'lastPrice']], on='Symbol', how='inner')

df_csv = pd.DataFrame(data)  # Assuming `data` is the read CSV result
df_merged = pd.merge(df_merged, df_csv[['Symbol', 'Sector', 'Quant Rating', 'Last Quarter Announce Date','EPS Revision Grade']], on='Symbol', how='left')

db_path = "plan.db"
query = """
SELECT 
    CValue.Symbol, 
    CValue.DCF, 
    CValue.Score AS Previous_Score,
    EMA169.note AS EMA_Note
FROM 
    CValue
LEFT JOIN 
    EMA169 ON CValue.Symbol = EMA169.symbol
WHERE 
    CValue.DCF IS NOT NULL;
"""

try:
    with sqlite3.connect(db_path) as conn:
        df_dcf_growth = pd.read_sql_query(query, conn)

    df_merged = pd.merge(df_merged, df_dcf_growth[['Symbol', 'DCF', 'Previous_Score','EMA_Note']], on='Symbol', how='left')
    df_merged['DCF'] = round(df_merged['DCF'].fillna('N'),2)  # Replace missing DCF values with 'N'
except Exception as e:
    print("Error fetching DCF data:", str(e))
    df_merged['DCF'] = 'N'  # Default to 'N' if database fetch fails
    df_merged['Previous_Score'] = None

# 保持原有计算
df_merged['LowDiff'] = df_merged.apply(
    lambda row: abs((row['lastPrice'] - row['LowPrice']) / row['lastPrice'] * 100)
    if row['LowPrice'] <= row['lastPrice'] <= row['HighPrice']
    else True if row['lastPrice'] < row['LowPrice']
    else None, axis=1
)

df_merged['HighDiff'] = df_merged.apply(
    lambda row: abs((row['lastPrice'] - row['HighPrice']) / row['lastPrice'] * 100)
    if row['LowPrice'] <= row['lastPrice'] <= row['HighPrice']
    else True if row['lastPrice'] > row['HighPrice']
    else None, axis=1
)

df_merged = df_merged.round(1)

# 保持原有Score计算
def calculate_growth_score(growth):
    if growth < 0:
        return -1
    elif 0 <= growth < 8:
        return 0
    elif 8 <= growth < 20:
        return 1
    else:
        return 2

def calculate_lowdiff_score(lowdiff, highdiff):
    if lowdiff is True:
        return 2
    elif isinstance(lowdiff, float):
        if lowdiff < 10 and lowdiff < highdiff:
            return 1
        elif 10 <= lowdiff < highdiff:
            return 0.5
        else:
            return 0
    elif lowdiff is None:
        return -1
    else:
        return 0

def calculate_quant_rating_score(rating):
    if rating > 4.5:
        return 1
    elif 3.5 <= rating <= 4.5:
        return 0.5
    elif 2.5 <= rating < 3.5:
        return 0
    else:
        return -1

def calculate_dcf_score(dcf, last_price,growth):
    if dcf == 'N':
        return -1.5
    try:
        if dcf < 0:
            return -1
        elif 0 < last_price < dcf * (1+growth/100):
            return 1
        else:
            return 0
    except TypeError:
        return 0
def calculate_PEG_score(PEG):    
    try:
        if PEG >= 2:
            return -0.5
        elif PEG <= 1.5:
            return 0.5
        else:
            return 0
    except:
        return -1
    
df_merged['Score'] = df_merged.apply(
    lambda row: calculate_growth_score(row['Growth']) +
                calculate_lowdiff_score(row['LowDiff'], row['HighDiff']) +
                calculate_quant_rating_score(row['Quant Rating']) +
                calculate_dcf_score(row['DCF'], row['lastPrice'],row['Growth']) + calculate_PEG_score(row['PEG']) + calculate_PEG_score(row['PEG5']),
    axis=1
)

# 输出分数变化的Symbol及分数
changed_scores = df_merged[df_merged.apply(
    lambda row: row['Score'] != row['Previous_Score'] if pd.notnull(row['Previous_Score']) else False,
    axis=1
)][['Symbol', 'Previous_Score', 'Score']]

if not changed_scores.empty:
    print("\nSymbols with score changes:")
    print(changed_scores.to_string(index=False))  # 打印所有分数变化的记录，去掉索引

# 排序
df_merged = df_merged.sort_values(by='Score', ascending=False)

# 准备展示的列（不包含 Previous_Score 和 Score_Changed）
df_display = df_merged[['Symbol', 'LowPrice', 'HighPrice', 'Growth', 'lastPrice', 'LowDiff', 'HighDiff', 'Quant Rating', 'DCF', 'Score', 'Sector', 'effective_epsy', 'Last Quarter Announce Date','EPS Revision Grade','PEG','PEG5','EMA_Note']]

# 修改样式函数
def highlight_row_with_score_diff(row):
    styles = [''] * len(row)

    # 原有样式保持
    styles = highlight_row(row)

    # 新增：Score变化高亮（如果需要打印的 Symbol）
    if row['Symbol'] in changed_scores['Symbol'].values:
        styles[row.index.get_loc('Score')] = 'background-color: yellow; color: black'
    # PEG styling
    if row['PEG'] > 1.5:
        styles[row.index.get_loc('PEG')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG')] = 'background-color: green'
     # PEG styling
    if row['PEG5'] > 1.5:
        styles[row.index.get_loc('PEG5')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG5')] = 'background-color: green'
    return styles

# 应用样式
styled_table = df_display.style.apply(highlight_row_with_score_diff, axis=1)

# 更新数据库
try:
    with sqlite3.connect(db_path) as conn:
        for _, row in df_merged.iterrows():
            conn.execute("""
                UPDATE CValue
                SET Score = ?
                WHERE Symbol = ?
            """, (row['Score'], row['Symbol']))
        conn.commit()
except Exception as e:
    print("Error updating scores in database:", str(e))

# 展示结果
styled_table


,Symbol,LowPrice,HighPrice,Growth,lastPrice,LowDiff,HighDiff,Quant Rating,DCF,Score,Sector,effective_epsy,Last Quarter Announce Date,EPS Revision Grade,PEG,PEG5,EMA_Note
218,SVM,3.100000,9.800000,66.700000,3.000000,True,None,4.100000,2.313395,6.500000,Materials,2026-03-31 00:00:00,11/7/2024,A,0.100000,1.100000,None
216,NGD,3.000000,15.200000,52.100000,2.700000,True,None,4.100000,14.231028,6.500000,Materials,2025-12-31 00:00:00,10/29/2024,A-,0.100000,0.200000,weekly up
168,SQ,91.800000,137.700000,22.600000,82.500000,True,None,4.500000,87.080292,6.500000,Financials,2025-12-31 00:00:00,11/7/2024,C+,0.800000,0.500000,weekly up
300,FMS,25.200000,64.500000,51.800000,22.500000,True,None,4.400000,86.965863,6.500000,Health Care,2025-12-31 00:00:00,11/5/2024,B+,0.200000,0.500000,short trend seq
187,LYFT,32.000000,42.700000,28.200000,13.000000,True,None,3.300000,79.224670,6.000000,Industrials,2025-12-31 00:00:00,11/6/2024,B+,0.400000,0.300000,short trend seq; down trend
230,MCY,117.300000,276.000000,8.700000,48.600000,True,None,4.800000,127.815615,6.000000,Financials,2025-12-31 00:00:00,10/29/2024,A+,1.000000,1.200000,down trend
243,DAR,36.500000,76.100000,30.400000,36.300000,True,None,2.700000,42.749887,6.000000,Consumer Staples,2025-12-31 00:00:00,10/24/2024,F,0.400000,0.700000,down trend
97,TRV,269.400000,434.800000,8.300000,232.400000,True,None,4.800000,1131.771040,6.000000,Financials,2025-12-31 00:00:00,10/17/2024,B+,1.400000,1.100000,weekly up
275,PDD,189.900000,316.400000,22.400000,94.700000,True,None,3.100000,2985.547409,6.000000,Consumer Discretionary,2025-12-31 00:00:00,11/21/2024,C-,0.400000,0.400000,None
165,UBER,69.300000,92.300000,42.000000,66.000000,True,None,3.400000,164.169020,6.000000,Industrials,2025-12-31 00:00:00,10/31/2024,A-,0.700000,0.700000,weekly up


In [8]:
from IPython.display import HTML, IFrame
import pandas as pd

# 生成一个带有排序和筛选功能的 HTML 文件
def render_datatable_to_file(df, output_file="table.html"):
    table_id = "data_table"
    html_content = f"""
    <html>
    <head>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.25/css/jquery.dataTables.css">
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
    <script src="https://cdn.datatables.net/1.10.25/js/jquery.dataTables.min.js"></script>
    </head>
    <body>
    <table id="{table_id}" class="display" style="width:100%">
        {df.to_html(index=False, classes="display")}
    </table>
    <script>
    $(document).ready(function() {{
        $('#{table_id}').DataTable({{
            "paging": fasle,          // 分页功能
            "searching": true,       // 搜索功能
            "ordering": true         // 排序功能
        }});
    }});
    </script>
    </body>
    </html>
    """
    # 保存 HTML 文件
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(html_content)

# 调用函数保存表格
render_datatable_to_file(df_display)

# 使用 IFrame 在 Notebook 中显示 HTML
IFrame(src="table.html", width="100%", height="500px")


In [12]:
# Add input for filtering by Sector
selected_sector = input("Enter the Sector to filter: ")

# Filter the DataFrame by the selected Sector
df_filtered = df_display[df_display['Sector'] == selected_sector]

# Highlight styling for the filtered DataFrame
def highlight_row(row):
    styles = [''] * len(row)

    # Growth column styling
    if row['Growth'] < 0:
        styles[row.index.get_loc('Growth')] = 'background-color: red; color: white'
    elif 0 <= row['Growth'] < 8:
        styles[row.index.get_loc('Growth')] = 'background-color: yellow'
    else:
        styles[row.index.get_loc('Growth')] = 'background-color: green; color: white'

    # LowDiff column styling
    if row['LowDiff'] is True:
        styles[row.index.get_loc('LowDiff')] = 'background-color: green; color: white'
    elif isinstance(row['LowDiff'], float):
        styles[row.index.get_loc('LowDiff')] = 'background-color: green' if row['LowDiff'] < row['HighDiff'] else 'background-color: yellow'
    elif row['LowDiff'] is None:
        styles[row.index.get_loc('LowDiff')] = 'background-color: red; color: white'

    # HighDiff column styling
    if row['HighDiff'] is True:
        styles[row.index.get_loc('HighDiff')] = 'background-color: red; color: white'
    elif isinstance(row['HighDiff'], float):
        styles[row.index.get_loc('HighDiff')] = 'background-color: green'
     
    # PEG styling
    if row['PEG'] > 1.5:
        styles[row.index.get_loc('PEG')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG')] = 'background-color: green'
     # PEG styling
    if row['PEG5'] > 1.5:
        styles[row.index.get_loc('PEG5')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG5')] = 'background-color: green'
        
    # Quant Rating styling
    if row['Quant Rating'] > 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: green; color: white'
    elif 3.5 < row['Quant Rating'] <= 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: yellow'
    elif 2.5 <= row['Quant Rating'] <= 3.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: lightgray'
    else:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: red; color: white'

    # DCF styling
    if row['DCF'] == 'N':
        styles[row.index.get_loc('DCF')] = 'background-color: lightgray'
    else:
        if row['lastPrice'] <= row['DCF'] * (1+row['Growth']/100):
            styles[row.index.get_loc('DCF')] = 'background-color: green; color: white'
        else:
            styles[row.index.get_loc('DCF')] = 'background-color: red; color: white'

    # Last Quarter Announce Date styling
    announce_date = pd.to_datetime(row['Last Quarter Announce Date'], errors='coerce')
    if pd.notnull(announce_date):
        # Range for ±7 days around Last Quarter Announce Date
        range_start_1 = announce_date - timedelta(days=8)
        range_end_1 = announce_date + timedelta(days=8)
        
        # Range for ±7 days around Last Quarter Announce Date + 90 days
        range_start_2 = announce_date + timedelta(days=90 - 8)
        range_end_2 = announce_date + timedelta(days=90 + 8)
        
        if range_start_1 <= today <= range_end_1:  # Within ±7 days of Last Quarter Announce Date
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: green; color: white'
        elif range_start_2 <= today <= range_end_2:  # Within ±7 days of Last Quarter Announce Date + 90 days
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: yellow'
    return styles

# Apply styling to the filtered DataFrame
styled_table = df_filtered.style.apply(highlight_row, axis=1)

# Display the styled DataFrame
styled_table
#Industrials,Utilities,Financials,Information Technology,Consumer Discretionary,Health Care,Communication Services,Consumer StaplesC

Enter the Sector to filter:  Utilities


,Symbol,LowPrice,HighPrice,Growth,lastPrice,LowDiff,HighDiff,Quant Rating,DCF,Score,Sector,effective_epsy,Last Quarter Announce Date,EPS Revision Grade,PEG,PEG5,EMA_Note
268,NRG,137.800000,275.700000,7.100000,99.400000,True,None,3.100000,99.252851,3.500000,Utilities,2025-12-31 00:00:00,11/8/2024,D+,1.900000,1.100000,weekly up
43,NEE,80.900000,103.000000,8.900000,67.400000,True,None,4.200000,79.948266,3.500000,Utilities,2025-12-31 00:00:00,10/23/2024,B+,2.200000,2.300000,None
159,EIX,70.000000,96.100000,6.500000,65.000000,True,None,3.700000,-165.830616,1.500000,Utilities,2025-12-31 00:00:00,10/29/2024,C,1.900000,1.600000,down trend
154,D,56.900000,71.100000,6.000000,53.400000,True,None,3.100000,-427.975053,1.000000,Utilities,2025-12-31 00:00:00,11/1/2024,C,2.700000,1.300000,short trend seq; down trend
228,SO,77.600000,129.300000,6.500000,81.300000,4.563035,59.061608,4.100000,-31.530318,-0.500000,Utilities,2025-12-31 00:00:00,10/31/2024,B+,2.900000,3.000000,weekly up
27,DUK,95.800000,125.600000,6.200000,105.900000,9.535211,18.622703,3.200000,-153.826481,-1.000000,Utilities,2025-12-31 00:00:00,11/7/2024,C-,2.800000,2.700000,weekly up


In [13]:
# Hardcoded list of symbols to filter by
selected_symbols = ['TRV', 'AMD', 'NVDA', 'LRCX','UNH','CI','MRK','ABT','HAS','TSN','BKR','SLB','VIST','PAAS','SVM','NGD','NEM','SMG','D','NEE','SCCO','TGB','DD','CTVA','UPS','GLDD','DAR','UBER','CTRA']  # Replace with your desired symbols

# Filter the DataFrame by the selected symbols
df_filtered = df_display[df_display['Symbol'].isin(selected_symbols)]

# Highlight styling for the filtered DataFrame
def highlight_row(row):
    styles = [''] * len(row)

    # Growth column styling
    if row['Growth'] < 0:
        styles[row.index.get_loc('Growth')] = 'background-color: red; color: white'
    elif 0 <= row['Growth'] < 8:
        styles[row.index.get_loc('Growth')] = 'background-color: yellow'
    else:
        styles[row.index.get_loc('Growth')] = 'background-color: green; color: white'

    # LowDiff column styling
    if row['LowDiff'] is True:
        styles[row.index.get_loc('LowDiff')] = 'background-color: green; color: white'
    elif isinstance(row['LowDiff'], float):
        styles[row.index.get_loc('LowDiff')] = 'background-color: green' if row['LowDiff'] < row['HighDiff'] else 'background-color: yellow'
    elif row['LowDiff'] is None:
        styles[row.index.get_loc('LowDiff')] = 'background-color: red; color: white'

    # HighDiff column styling
    if row['HighDiff'] is True:
        styles[row.index.get_loc('HighDiff')] = 'background-color: red; color: white'
    elif isinstance(row['HighDiff'], float):
        styles[row.index.get_loc('HighDiff')] = 'background-color: green'
     
    # PEG styling
    if row['PEG'] > 1.5:
        styles[row.index.get_loc('PEG')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG')] = 'background-color: green'
     # PEG styling
    if row['PEG5'] > 1.5:
        styles[row.index.get_loc('PEG5')] = 'background-color: red; color: white'
    elif isinstance(row['PEG'], float):
        styles[row.index.get_loc('PEG5')] = 'background-color: green'
        
    # Quant Rating styling
    if row['Quant Rating'] > 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: green; color: white'
    elif 3.5 < row['Quant Rating'] <= 4.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: yellow'
    elif 2.5 <= row['Quant Rating'] <= 3.5:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: lightgray'
    else:
        styles[row.index.get_loc('Quant Rating')] = 'background-color: red; color: white'

    # DCF styling
    if row['DCF'] == 'N':
        styles[row.index.get_loc('DCF')] = 'background-color: lightgray'
    else:
        if row['lastPrice'] <= row['DCF'] * (1+row['Growth']/100):
            styles[row.index.get_loc('DCF')] = 'background-color: green; color: white'
        else:
            styles[row.index.get_loc('DCF')] = 'background-color: red; color: white'

    # Last Quarter Announce Date styling
    announce_date = pd.to_datetime(row['Last Quarter Announce Date'], errors='coerce')
    if pd.notnull(announce_date):
        # Range for ±7 days around Last Quarter Announce Date
        range_start_1 = announce_date - timedelta(days=8)
        range_end_1 = announce_date + timedelta(days=8)
        
        # Range for ±7 days around Last Quarter Announce Date + 90 days
        range_start_2 = announce_date + timedelta(days=90 - 8)
        range_end_2 = announce_date + timedelta(days=90 + 8)
        
        if range_start_1 <= today <= range_end_1:  # Within ±7 days of Last Quarter Announce Date
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: green; color: white'
        elif range_start_2 <= today <= range_end_2:  # Within ±7 days of Last Quarter Announce Date + 90 days
            styles[row.index.get_loc('Last Quarter Announce Date')] = 'background-color: yellow'
    return styles

# Apply styling to the filtered DataFrame
styled_table = df_filtered.style.apply(highlight_row, axis=1)

# Display the styled DataFrame
styled_table

,Symbol,LowPrice,HighPrice,Growth,lastPrice,LowDiff,HighDiff,Quant Rating,DCF,Score,Sector,effective_epsy,Last Quarter Announce Date,EPS Revision Grade,PEG,PEG5,EMA_Note
218,SVM,3.100000,9.800000,66.700000,3.000000,True,None,4.100000,2.313395,6.500000,Materials,2026-03-31 00:00:00,11/7/2024,A,0.100000,1.100000,None
216,NGD,3.000000,15.200000,52.100000,2.700000,True,None,4.100000,14.231028,6.500000,Materials,2025-12-31 00:00:00,10/29/2024,A-,0.100000,0.200000,weekly up
243,DAR,36.500000,76.100000,30.400000,36.300000,True,None,2.700000,42.749887,6.000000,Consumer Staples,2025-12-31 00:00:00,10/24/2024,F,0.400000,0.700000,down trend
97,TRV,269.400000,434.800000,8.300000,232.400000,True,None,4.800000,1131.771040,6.000000,Financials,2025-12-31 00:00:00,10/17/2024,B+,1.400000,1.100000,weekly up
165,UBER,69.300000,92.300000,42.000000,66.000000,True,None,3.400000,164.169020,6.000000,Industrials,2025-12-31 00:00:00,10/31/2024,A-,0.700000,0.700000,weekly up
293,CI,504.800000,788.700000,11.300000,279.000000,True,None,3.800000,606.805470,5.500000,Health Care,2025-12-31 00:00:00,10/31/2024,C-,0.800000,0.800000,None
172,HAS,69.300000,129.400000,12.400000,57.000000,True,None,4.000000,152.270710,5.500000,Consumer Discretionary,2025-12-31 00:00:00,10/24/2024,B+,1.000000,1.100000,short trend seq; down trend
267,CTRA,14.500000,116.300000,25.300000,27.800000,47.666547,318.667627,3.600000,40.395758,5.000000,Energy,2025-12-31 00:00:00,10/31/2024,C,0.400000,0.900000,weekly up
140,AMD,153.000000,204.000000,38.200000,116.000000,True,None,3.100000,67.771629,5.000000,Information Technology,2025-12-31 00:00:00,10/29/2024,D+,0.600000,0.700000,weekly up
142,NVDA,132.900000,177.200000,50.000000,135.900000,2.195276,30.406298,3.500000,109.578748,5.000000,Information Technology,2026-01-31 00:00:00,11/20/2024,B+,0.900000,1.800000,weekly up
